In [5]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state,get_dropbox_folders


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plates = [
"1_20220406",
"2_20220411",
"4_20220419",
"16_20220419",
"20_20220419",
"832_20220419",
"871_20220419",
"877_20220419",
"13_20220422",
"18_20220422",
"3_20220426",
"24_20220426",
"87_20220611",
"85_20220706",
"88_20220708",
"91_20220708",
"82_20220711",

]
# plates = ["599_20201217"]
# plates = [
#     "799_20211001",
#     "803_20211012",
#     "809_20211012",
#     "907_20220321",
#     "943_20220322",
#     "403_20221005",
#     "408_20221013",
#     "88_20210915",
#     "790_20210907",
#     "69_20201119",
#     "94_20201123",
#     "59_20201126",
#     "599_20201217",
#     "139_20201214",
#     "40_20201214"

    
# ]
# plates = [
# # '771_20230411',
# # '772_20230317',
# # '782_20230406',
# # '784_20230324',
# # '792_20230324',
# '805_20230311',
# # '822_20230401',
# # '828_20230330'
# ]

# plates = [
#     "52_20220707",
#     "56_20220701",
#     "59_20201126",
#     "69_20201119",
#     "76_20210108",
#     "81_20220704",
#     "84_20220621",
#     "94_20201123",
#     "102_20201226",
#     "152_20201224",
#     "792_20210915",
#     "1076_20220511",
#     "941_20220404"
# ]
plates = [
"211_20230422",
"212_20230228",
"214_20230309",
"217_20230415",
"218_20230227",
"219_20230307",
"220_20230404",
"221_20230427",
"222_20230324",
"224_20230227",
"226_20230228",
"227_20230418",
"228_20230401",
"229_20230330",
"230_20230313",
"232_20230321",
"236_20230313",
"237_20230307",
"239_20230411",
"240_20230328",
"241_20230403",
"242_20230307",
"245_20230301",
"246_20230316",
"247_20230422",
"248_20230320",
"249_20230324",
"250_20230508",
"252_20230316",
"254_20230327",
"255_20230425",
"256_20230307",
"257_20230421"
]

In [14]:
plates = [
"818_20220301",
# "824_20220413",
"839_20220325",
"845_20220222",
"854_20220222",
"860_20220317",
# "866_20220315",
"875_20220301",
"887_20220330",
"890_20220608",
"926_20220304",
"938_20220325",
"941_20220404",
"953_20220301",
"977_20220304",
"983_20220321",
"986_20220225",
"992_20220225",
"1001_20220304",
"1007_20220301"

]
# plates = [
# "824_20220414",
# "1015_20220504",
# "1019_20220419",
# "1023_20220506",
# "1025_20220518",
# "1027_20220607",
# "1031_20220419",
# "1033_20220502",
# "1035_20220518",
# "1037_20220523",
# "1039_20220516",
# "1045_20220504",
# "1047_20220516",
# "1051_20220518",
# "1053_20220413",
# "1070_20220502",
# "1071_20220517",
# "1076_20220511"
# ]

# plates = [
# "41_20220708",
# "43_20220718",
# "49_20221029",
# "51_20220728",
# "59_20220715",
# "63_20220704",
# "67_20220923",
# "71_20220817",
# "73_20220630",
# "140_20220826",
# "144_20220919"
    
# ]
# plates = ['1048_20220506','32_20220428', '33_20220502', '52_20220707','84_20220621', '91_20220708', '938_20220325', "1014_20220525",
#     "1023_20220502",
#     "1042_20220523",
#     "1045_20220504",
#     "1053_20220413",
#     "1076_20220511"]

In [2]:
plates = ['907_20220321']

In [2]:
plates = [
    "52_20220707",
    "56_20220701",
    "59_20201126",
    "69_20201119",
    "76_20210108",
    "81_20220704",
    "84_20220621",
    "94_20201123",
    "102_20201226",
    "152_20201224",
    "792_20210915",
    "1076_20220511",
    "941_20220404"
]


In [47]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    50,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="kaikai",
)

In [3]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-12-15 19:07:38,275-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-15 19:07:38,564-[INFO]- dropbox:474 -> Request to files/list_folder


KeyboardInterrupt: 

In [5]:
dir_drop = "DATA/PRINCE"
all_folders_drop = read_saved_dropbox_state("/DATA/PRINCE")
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
update_plate_info(directory_targ, local=True, strong_constraint=False)
all_folders = get_current_folders(directory_targ, local=True)
# if len(all_folders) > 0:
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders_drop2 = folders_drop.loc[~folders_drop["folder"].isin(folders["folder"])]
folders_drop3 = folders_drop2.loc[~folders_drop2["folder"].str.contains("Analysis")]

2023-05-22 09:19:19,831-[INFO]- dropbox:390 -> Refreshing access token.
2023-05-22 09:19:19,974-[INFO]- dropbox:474 -> Request to files/download


analysed:   0%|          | 0/5119 [00:00<?, ?it/s]

In [6]:
folders_drop3

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
0,20230313_1443_Plate07,NaN,0,data/prince/218_20230227/20230313_1443_plate07,218,20230227,218_20230227


In [6]:
# plates= ['94_20201123','102_20201226','76_20210108','152_20201224','792_20210915']
# plates= ['52_20220707','56_20220701','839_20220325','938_20220325','1076_20220511']
plates = [
    "52_20220707",
    "56_20220701",
    "81_20220704",
    "89_20220705",
    "91_20220708",
    "839_20220325",
    "1076_20220511",
]  # full run done!
plates = [
    "84_20220621",
    "877_20220419",
    "938_20220325",
    "941_20220404",
]  # for spore data

# plates = ['52_20220707','84_20220621','1048_20220506','1045_20220504']
plates = [
    # "20_20220419",
    # "23_20220509",
    # "24_20220426",
    # "30_20220509",
    # "32_20220428",
    "33_20220502",
]  # analysis AGG
# plates = ["20_20220419", "23_20220509", "24_20220426"]  # analysis AGG

# plates = ['111111_20600101']
# plates = ["56_20220701","839_20220325",'938_20220325','1076_20220511'] #analysis A5
# plates = ["81_20220704","84_20220621","89_20220705","91_20220708"] #analysis A5sp3
# plates = ["127_20220705","129_20220711","130_20220707","1014_20220525","1023_20220502","1048_20220506", "1045_20220504","1035_20220518","1037_20220523","1042_20220523","1053_20220413"] #analysis C2
# plates = ["1035_20220518","1037_20220523","1042_20220523","1053_20220413"] #analysis C2
# plates = ['94_20201123','152_20201224','792_20210915']
# plates = ['799_20211001','800_20210928','91_20201217']
# plates = ["1044_20220509"]
# plates = ["1048_20220506", "1045_20220504", "52_20220707", "84_20220621"]
# plates = [
#     '102_20201226',
#      '94_20201123',
#     "1076_20220511",
#     "1042_20220523",
#     "1048_20220506",
#     "938_20220325",
#          ]
# plates = ['907_20220321','943_20220322',"403_20221005"] #for philippe
# plates = ['799_20211001','800_20210928',"91_20201217"]
plates = [
    "106_20220826",
    "108_20220823",
    "111_20220823",
    "66_20220823",
    "105_20220817",
    "100_20220812",
    "75_20220808",
    "99_20220815",
]

plates = ["521_20230104"]  # for spore data
# plates = ['1048_20220506','84_20220621'] #for spore data
plates = [
    "799_20211001",
    "803_20211012",
    "809_20211012",
    "907_20220321",
    "943_20220322",
    "403_20221005",
    "408_20221013",
    "88_20210915",
    "790_20210907",
    "69_20201119",
    "94_20201123",
    "59_20201126",
    "599_20201217",
    "139_20201214",
    "40_20201214"
]

In [2]:
plates = [
    "078_20220920",
    "079_20220920",
    "080_20220920",
    "081_20220920",
    "082_20220920",
    "083_20220920",
    "084_20220920",
    "001_20220920",
    "024_20220920",
    "073_20220920",
    "002_20220920",
    "003_20220920",
    "004_20220920",
    "005_20220920",
    "006_20220920",
    "007_20220920",
    "008_20220920",
    "009_20220920",
    "010_20220920",
    "011_20220920",
    "012_20220920",
    "019_20220920",
    "020_20220920",
    "021_20220920",
    "022_20220920",
    "023_20220920",
    "037_20220920",
    "038_20220920",
    "039_20220920",
    "040_20220920",
    "042_20220920",
    "043_20220920",
    "044_20220920",
    "045_20220920",
    "046_20220920",
    "016_20220921",
    "017_20220921",
    "034_20220921",
    "051_20220920",
    "094_20220921",
    "064_20220921",
    "026_20220921",
    "027_20220921",
    "028_20220921",
    "029_20220921",
    "031_20220921",
    "033_20220921",
    "035_20220921",
    "036_20220921",
    "047_20220920",
    "041_20220920",
    "015_20220921",
    "068_20220921",
    "107_20220921",
    "095_20220921",
    "062_20220921",
    "096_20220921",
    "104_20220921",
    "050_20220920",
    "030_20220921",
    "025_20220921",
    "100_20220921",
    "067_20220921",
    "048_20220920",
    "055_20220921",
    "056_20220921",
    "057_20220921",
    "058_20220921",
    "059_20220921",
    "060_20220921",
    "061_20220921",
    "063_20220921",
    "014_20220921",
    "018_20220921",
    "099_20220921",
    "069_20220921",
    "054_20220920",
    "075_20220920",
    "092_20220921",
    "093_20220921",
    "097_20220921",
    "098_20220921",
    "105_20220921",
    "106_20220921",
    "108_20220921",
    "085_20220921",
    "086_20220921",
    "087_20220921",
    "088_20220921",
    "089_20220921",
    "090_20220921",
    "091_20220921",
    "049_20220920",
    "052_20220920",
    "053_20220920",
    "074_20220920",
    "076_20220920",
    "077_20220920",
    "013_20220921",
    "070_20220921",
    "071_20220921",
    "072_20220921",
    "102_20220921",
    "032_20220921",
    "101_20220921",
    "065_20220921",
    "066_20220921",
    "103_20220921",
]
plates = [
    "_".join((str(int(plate.split("_")[0])), str(int(plate.split("_")[1]))))
    for plate in plates
]

In [3]:
directory_targ = directory_project
directory_project

'/projects/0/einf914/data/'

In [3]:
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"

In [4]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, -1, "stitcher.py"],
    plates,
    "12:00:00",
    name_job=name_job,
    dependency=True,
)
# run_launcher('dropbox_uploader.py',[directory_targ,name_job],plates,'40:00',dependency=True,name_job = name_job)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2784310


In [5]:
name_job = f"one_shot_{time_ns()}.sh"
name_job = "table_redo"
run_launcher(
    "dropbox_downloader.py",
    [directory_targ, name_job, 30, "post_processer_1.py"],
    plates,
    "24:00:00",
    name_job=name_job,
    dependency=False,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2397930


In [7]:
name_job = "job2.sh"

# name_job = f"one_shot_{time_ns()}.sh"

run_launcher(
    "stitcher.py",
    [directory_targ, name_job, 1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2781039


In [6]:
update_analysis_info(directory_targ)

analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [18]:
d

NameError: name 'd' is not defined

In [18]:
name_job = "recompute"

run_launcher(
    "hypha_identifier.py",
    [directory_targ, name_job, 10],
    plates,
    "24:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2663449


In [5]:
name_job = "density2"

run_launcher(
    "dropbox_uploader.py",
    [directory_targ, name_job, -1],
    plates,
    "5:00:00",
    dependency=False,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2719443


In [4]:
# name_job = "recompute"

# name_job = f"one_shot_{time_ns()}.sh"
name_job = "one_shot_1675411679561103361"
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job, -1],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2693614


In [8]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

DATA/PRINCE
2022-12-14 15:38:52,796-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 15:38:53,186-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 15:38:54,631-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 15:38:54,971-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 15:38:56,828-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 15:38:57,077-[INFO]- dropbox:474 -> Request to files/upload


Uploaded:   0%|          | 0/141529031 [00:00<?, ?it/s]

2022-12-14 15:38:58,762-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 15:38:59,097-[INFO]- dropbox:474 -> Request to files/upload_session/start
2022-12-14 15:38:59,990-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:01,179-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:02,417-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:03,681-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:05,039-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:06,314-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:07,268-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:08,492-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:09,701-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:10,784-[INFO]- dropbox:474 -> Request to files/uploa

Uploaded:   0%|          | 0/13295488 [00:00<?, ?it/s]

2022-12-14 15:39:49,467-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 15:39:49,741-[INFO]- dropbox:474 -> Request to files/upload_session/start
2022-12-14 15:39:51,229-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:52,248-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 15:39:53,736-[INFO]- dropbox:474 -> Request to files/upload_session/finish


...

2022-12-14 16:44:54,916-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:44:55,166-[INFO]- dropbox:474 -> Request to files/upload_session/start
2022-12-14 16:44:56,064-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:44:57,605-[INFO]- dropbox:474 -> Request to files/upload_session/finish
2022-12-14 16:44:59,176-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:44:59,632-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:45:00,787-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:45:01,038-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:45:02,657-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:45:02,908-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:45:04,946-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:45:05,211-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:45:06,561-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:45:06,880-[IN

Uploaded:   0%|          | 0/68200392 [00:00<?, ?it/s]

2022-12-14 16:48:57,867-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:48:58,114-[INFO]- dropbox:474 -> Request to files/upload_session/start
2022-12-14 16:48:59,638-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:01,135-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:02,977-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:04,585-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:05,869-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:06,972-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:08,170-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:09,267-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:10,465-[INFO]- dropbox:474 -> Request to files/upload_session/append
2022-12-14 16:49:11,585-[INFO]- dropbox:474 -> Request to files/uploa

Uploaded:   0%|          | 0/6921621 [00:00<?, ?it/s]

2022-12-14 16:49:22,015-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:22,250-[INFO]- dropbox:474 -> Request to files/upload_session/start
2022-12-14 16:49:23,080-[INFO]- dropbox:474 -> Request to files/upload_session/finish
2022-12-14 16:49:24,955-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:25,247-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:49:27,040-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:27,287-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:49:29,111-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:29,345-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:49:31,108-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:31,360-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:49:32,699-[INFO]- dropbox:390 -> Refreshing access token.
2022-12-14 16:49:32,924-[INFO]- dropbox:474 -> Request to files/upload
2022-12-14 16:49:34,347-[INFO]- dropbox:39

In [16]:
name_job = f"one_shot_{time_ns()}.sh"
run_launcher(
    "width_extracter.py",
    [directory_targ, name_job, 20],
    plates,
    "20:00",
    dependency=False,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2252534


In [5]:
import sys

import pandas as pd

from amftrack.util.sys import temp_path
from amftrack.util.dbx import download_folders_drop


op_id = 1674918618515998552
run_info = pd.read_json(f"{temp_path}/{op_id}.json", dtype={"unique_id": str})

In [13]:
len(run_info)

46

In [28]:
import pandas as pd
# plate_info = get_data_info(True, "")
source = f"/data_info.json"
target = os.path.join(temp_path, f"data_info{''}.json")

data_info = pd.read_json(target, convert_dates=True).transpose()
if len(data_info) > 0:
    data_info.index.name = "total_path"
    data_info.reset_index(inplace=True)
data_info.loc[data_info["Plate"].astype(str)=="nan"]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder
979,/projects/0/einf914/data/20220508_1411_Plate10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,"08.05.2022, 14:11:",20220508_1411_Plate10


In [26]:
data_info["Plate"].astype(str)

0         87
1         85
2         24
3          4
4         13
        ... 
3431      24
3432      16
3433      24
3434      16
3435    1053
Name: Plate, Length: 3436, dtype: object

In [30]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
# folders =all_folders.loc[all_folders["unique_id"].isin(plates)]
# folders = folders.loc[folders["/Analysis/skeleton_pruned_compressed.mat"]==True]

analysed:   0%|          | 0/3712 [00:00<?, ?it/s]

In [34]:
folders = all_folders.loc[~all_folders["unique_id"].isin(plates)]

len(folders)

2551

In [36]:
import shutil
for folder in folders["total_path"]:
    shutil.rmtree(folder)
    print(folder)
    # break

/projects/0/einf914/data/20220717_1252_Plate03
/projects/0/einf914/data/20220711_0338_Plate03
/projects/0/einf914/data/20220709_1138_Plate03
/projects/0/einf914/data/20220715_1452_Plate03
/projects/0/einf914/data/20220903_1800_Plate10
/projects/0/einf914/data/20220926_2037_Plate10
/projects/0/einf914/data/20201116_0839_Plate32
/projects/0/einf914/data/20220828_0253_Plate13
/projects/0/einf914/data/20220903_0159_Plate10
/projects/0/einf914/data/20220718_2209_Plate14
/projects/0/einf914/data/20221019_2216_Plate07
/projects/0/einf914/data/20221003_1104_Plate34
/projects/0/einf914/data/20220929_1233_Plate07
/projects/0/einf914/data/20221003_0305_Plate34
/projects/0/einf914/data/20220806_0028_Plate32
/projects/0/einf914/data/20220714_2028_Plate12
/projects/0/einf914/data/20220827_1453_Plate13
/projects/0/einf914/data/20210918_1729_Plate21
/projects/0/einf914/data/20220909_1550_Plate13
/projects/0/einf914/data/20220701_0710_Plate05
/projects/0/einf914/data/20220918_1814_Plate13
/projects/0/e

In [35]:
plates = all_folders['unique_id'].unique()
plates

array(['1019_20220419', '1031_20220419', '63_20220704', '1051_20220518',
       '1053_20220413', '1023_20220506', '1035_20220518', '1071_20220517',
       '1045_20220504', '1070_20220502', '1027_20220607', '71_20220817',
       '144_20220919', '152_20201224', '140_20220826', '43_20220718',
       '1076_20220511', '67_20220923', '1025_20220518', '1037_20220523',
       '59_20220715', '81_20220704', '792_20210915', '1047_20220516',
       '73_20220630', '824_20220414', '1033_20220502', '1039_20220516',
       '1015_20220504', '941_20220404', '102_20201226', '1042_20220523',
       '30_20220509', '76_20210108', '800_20210928', '32_20220428',
       '1023_20220502', '59_20201126', '907_20220321', '56_20220701',
       '52_20220707', '94_20201123', '33_20220502'], dtype=object)

In [6]:
plates = [plate for plate in all_folders["unique_id"].unique() if plate not in plates]

In [20]:
num_parallel = 100
time = "6:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
)

Submitted batch job 2277244


In [11]:
name_job = "empty"
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    50,
    "24:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2740047


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2740048


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 2740091


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2740092


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2740093


In [10]:
all_folders["unique_id"].unique()

array(['69_20201119', '753_20221013', '764_20221219', '765_20221213',
       '88_20210915', '790_20210907', '757_20221019', '731_20221013',
       '809_20211012', '737_20221025', '799_20211001', '762_20221213',
       '763_20230109', '408_20221013', '734_20221019', '803_20211012',
       '752_20221025', '907_20220321', '403_20221005', '40_20201214',
       '943_20220322'], dtype=object)

In [7]:
folders.loc[folders["/Analysis/transform.mat"] == True]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
18,/projects/0/einf914/data/20230106_1749_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"06.01.2023, 17:49:",20230106_1749_Plate02,NaN,521_20230104,2023-01-06 17:49:00
25,/projects/0/einf914/data/20230125_0554_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"25.01.2023, 05:54:",20230125_0554_Plate02,NaN,521_20230104,2023-01-25 05:54:00
38,/projects/0/einf914/data/20230110_1546_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"10.01.2023, 15:46:",20230110_1546_Plate02,NaN,521_20230104,2023-01-10 15:46:00
78,/projects/0/einf914/data/20230123_1554_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"23.01.2023, 15:54:",20230123_1554_Plate02,NaN,521_20230104,2023-01-23 15:54:00
101,/projects/0/einf914/data/20230124_2154_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"24.01.2023, 21:54:",20230124_2154_Plate02,NaN,521_20230104,2023-01-24 21:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,/projects/0/einf914/data/20230112_0549_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"12.01.2023, 05:49:",20230112_0549_Plate02,NaN,521_20230104,2023-01-12 05:49:00
1685,/projects/0/einf914/data/20230109_1409_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"09.01.2023, 14:09:",20230109_1409_Plate02,NaN,521_20230104,2023-01-09 14:09:00
1700,/projects/0/einf914/data/20230105_0833_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,True,True,True,True,"05.01.2023, 08:33:",20230105_0833_Plate02,NaN,521_20230104,2023-01-05 08:33:00
1737,/projects/0/einf914/data/20230124_0954_Plate02,521,2,'Carrot','C2','001P100N100C','Y','-',20230104,25,...,False,False,False,False,False,"24.01.2023, 09:54:",20230124_0954_Plate02,NaN,521_20230104,2023-01-24 09:54:00


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"
stage = 30
args = [directory_targ]
for plate in plates:
    select = folders.loc[folders["unique_id"] == plate]
    num_parallel = 32
    time = "10:00"
    args = [directory_targ]
    run_parallel(
        "realign.py",
        args,
        select,
        num_parallel,
        time,
        "create_realign",
        cpus=32,
        node="fat",
        name_job=name_job,
    )

run_launcher(
    "graph_extracter.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872208


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872209


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


...

Submitted batch job 1872314


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1872315


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1872316


In [7]:
name_job = "rerun"
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, 10],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2143049


In [8]:
from amftrack.util.dbx import upload_folder

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
dir_drop = "DATA/PRINCE"

for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    id_unique = row["unique_id"]
    path = os.path.join(directory_targ, folder)
    target_drop = f"/{dir_drop}/{id_unique}/{folder}"
    print(dir_drop)
    upload_folder(path, target_drop, delete=False)

/projects/0/einf914/data/Analysis_93_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_5_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_69_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_26_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_32_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_43_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_15_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_65_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_28_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0

In [16]:
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import run_parallel_post

name_job = "malin"
time = "1:00:00"
list_f = [plot_tracking]
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ
list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 100
run_parallel_post(
    "exp_plot.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_plate_post_process",
    cpus=128,
    name_job=name_job,
    node="fat",
)

/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_67_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_56_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_50_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_23_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_21_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_100_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_82_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_61_20220921_200_Version1/folder_info.json
/projects

In [15]:
from amftrack.pipeline.functions.post_processing.time_hypha import *
import pandas as pd
import os

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    get_has_reached_final_pos,
    get_in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "10:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

/projects/0/einf914/data/Analysis_101_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_42_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_39_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_108_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_67_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_56_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_50_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_23_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_21_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_100_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_82_20220920_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_61_20220921_200_Version1/folder_info.json
/projects

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867673
Submitted batch job 1867674


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867676


...

Submitted batch job 1867715


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867716


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1867717


In [36]:
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
args = [directory_targ]
num_parallel = 32
time = "1:00:00"
limit = 200
version = 1
suffix = "_labeled"
lim_considered = 10
args = [directory_targ, limit, version, suffix, lim_considered]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
    name_job="test",
)

Submitted batch job 1857991


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857992


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857995


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1857997


In [14]:
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
len(folders)

451

In [20]:
len(list(set(folders["unique_id"].values)))

107

In [12]:
stage = 20
run_launcher(
    "post_processer_1.py",
    [directory_targ, name_job, stage - 1],
    plates,
    "20:00",
    dependency=True,
    name_job="test",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2061667


In [28]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

/projects/0/einf914/data/Analysis_93_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_92_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_95_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_92_20220921_4_Version1/folder_info.json
/projects/0/einf914/data/Analysis_99_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analysis_95_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_94_20220921_1_Version1/folder_info.json
/projects/0/einf914/data/Analysis_93_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_90_20220921_2_Version1/folder_info.json
/projects/0/einf914/data/Analysis_98_20220921_3_Version1/folder_info.json
/projects/0/einf914/data/Analysis_91_20220921_200_Version1/folder_info.json
/projects/0/einf914/data/Analy

In [7]:
analysis_folders

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,datetime,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis
0,Analysis_33_20220502_188_Version1,Version1,/projects/0/einf914/data/20220502_2111_Plate07,33,7,'Carrot','Agg','001P100N100C','Y','-',...,2022-05-02 21:11:00,"02.05.2022, 21:11:","20.05.2022, 09:41:",188,Analysis_33_20220502_188_Version1/experiment.pick,Analysis_33_20220502_188_Version1/global_hypha...,Analysis_33_20220502_188_Version1/time_hypha_info,Analysis_33_20220502_188_Version1/time_plate_i...,Analysis_33_20220502_188_Version1/global_plate...,"01.01.1970, 01:00:"


In [8]:
# name_job = f"one_shot_{time_ns()}.sh"

from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

time = "24:00:00"
directory = directory_targ
max_ind = 20
incr = 100
fs = [
    get_density_in_ring,
    get_density_anastomose_in_ring,
    # get_density_branch_rate_in_ring,
    # get_density_stop_rate_in_ring,
    get_density_active_tips_in_ring,
]
# fs = [get_mean_speed_in_ring]

list_f = []
list_args = []

for f in fs:
    list_f += [f] * max_ind

    list_args += [{"incr": incr, "i": i, "rh_only": True} for i in range(max_ind)]
overwrite = False
num_parallel = 30
run_parallel_post(
    "time_plate_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "time_plate_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
    dependency=True,
)
run_launcher(
    "analysis_uploader.py",
    [directory_targ, name_job],
    plates,
    "12:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1768604


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1768605


In [41]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

name_job = "malin"

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "compress_image.py",
    args,
    folders,
    num_parallel,
    time,
    "compress",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858250
Submitted batch job 1858251
Submitted batch job 1858252
Submitted batch job 1858253
Submitted batch job 1858255
Submitted batch job 1858256


In [42]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 50
time = "12:00:00"
hyph_width = 30
perc_low = 93
perc_high = 99.5
args = [hyph_width, perc_low, perc_high, directory_targ]
run_parallel(
    "extract_skel_2.py",
    args,
    folders,
    num_parallel,
    time,
    "skeletonization",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858259
Submitted batch job 1858260
Submitted batch job 1858262
Submitted batch job 1858267
Submitted batch job 1858268
Submitted batch job 1858269
Submitted batch job 1858270
Submitted batch job 1858271
Submitted batch job 1858272
Submitted batch job 1858273
Submitted batch job 1858274
Submitted batch job 1858275


In [43]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

num_parallel = 100
time = "20:00"
args = [directory_targ]
run_parallel(
    "detect_blob.py",
    args,
    folders,
    num_parallel,
    time,
    "detect_blob",
    cpus=128,
    node="fat",
    name_job=name_job,
)

Submitted batch job 1858277
Submitted batch job 1858279
Submitted batch job 1858282
Submitted batch job 1858283
Submitted batch job 1858284
Submitted batch job 1858285


In [7]:
name_job = "malin"

run_launcher(
    "masker.py",
    [directory_targ, name_job, 30],
    plates,
    "3:00:00",
    dependency=True,
    name_job=name_job,
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


Submitted batch job 1861160


In [25]:
from amftrack.pipeline.launching.run_super import run_parallel_all_time, run_launcher

folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Img/TileConfiguration.txt.registered"] == True]
num_parallel = 50
time = "10:00"
args = []
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=False,
    name_job="video",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828129


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828130


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1828131


In [3]:
plates = [
    "105_20220817",
    "77_20220808",
    "71_20220817",
    "100_20220812",
    "117_20220815",
    "99_20220815",
    "75_20220808",
    "226_20220812",
]

In [46]:
folders = all_folders.loc[all_folders["Plate"].isin(["32"])]

In [17]:
from amftrack.pipeline.launching.run_super import *

num_parallel = 32
time = "8:00:00"
limit = 200
version = 1
labeled = True
args = [directory_targ, limit, version, labeled]
run_parallel_all_time(
    "hyphae_extraction.py",
    args,
    folders,
    num_parallel,
    time,
    "hyphae",
    cpus=32,
    node="fat",
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1738329


In [12]:
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher

for plate in plates:
    select = folders.loc[folders["Plate"] == plate]
    num_parallel = 128
    time = "12:00:00"
    thresh = 10000  # For R. irregularis, thresh 10000 is good. For Aggregatum, higher may be necessary
    args = [thresh, directory_targ]
    run_parallel(
        "final_alignment.py",
        args,
        select,
        num_parallel,
        time,
        "realign",
        cpus=128,
        node="fat",
        name_job=name_job,
    )

Submitted batch job 1138954


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [20]:
plates

['6_20220509']

In [19]:
select

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


In [13]:
i = 0
run_info = folders
folder_list = list(run_info["folder"])
folder_list.sort()

dates_datetime_chosen = folder_list[i : i + 2]
print("========")
print(f"Matching plate {dates_datetime_chosen[0]} at dates {dates_datetime_chosen}")
print("========")
dates = dates_datetime_chosen

Matching plate 20220510_1851_Plate17 at dates ['20220510_1851_Plate17', '20220511_1750_Plate17']


In [10]:
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
from amftrack.pipeline.functions.post_processing.exp_plot import *

update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]
directory = directory_targ

In [11]:
time = "3:40:00"
directory = directory_targ
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [
    gets_out_of_ROI,
    get_width_f,
    get_tot_length_C_f,
    get_tot_growth_C_f,
    get_tot_length_pp_f,
    get_tot_growth_pp_f,
    get_timestep_stop_growth,
    get_time_stop_growth,
    get_time_init_growth,
    get_mean_speed_growth,
    get_stop_track,
    get_timestep_anastomosis,
    get_timestep_biological_stop_growth,
]
# list_f = [get_num_branch]
# list_f = [gets_out_of_ROI]
list_args = [{}] * len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post(
    "global_hypha_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_hypha_post_process",
    cpus=32,
    name_job="a",
    node="fat",
)

Submitted batch job 1539552


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [13]:
import sys
from amftrack.util.sys import (
    update_analysis_info,
    get_analysis_info,
)
from amftrack.pipeline.launching.run_super import run_parallel_post
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.launching.run_super import run_parallel, run_launcher
import pandas as pd
import os

list_f = [
    get_pos_x,
    get_pos_y,
    get_time_since_begin_exp,
    get_distance_final_pos,
    get_timedelta,
    get_time_since_start,
    get_speed,
    get_timestep,
    get_timestep_init,
    get_time_init,
    get_degree,
    get_width_tip_edge,
    get_width_root_edge,
    get_width_average,
    has_reached_final_pos,
    in_ROI,
]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
# list_f = [get_width_tip_edge, get_width_root_edge]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 32
time = "12:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_hypha_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "time_hypha_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )

Submitted batch job 1539625
Submitted batch job 1539626
Submitted batch job 1539627


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memor

Submitted batch job 1539628
Submitted batch job 1539629
Submitted batch job 1539630


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539631


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539645
Submitted batch job 1539646


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1539647
Submitted batch job 1539648
Submitted batch job 1539649


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
